In [2]:
# importing neccesary libaries
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from MassProfile import MassProfile

# for contours
import scipy.optimize as so

In [4]:
# identifying simulation snapshots
# From the previous seperation plots, we need - 4 Gyr, 6 Gyr and a snapshot after this close encounter
# eg. 4 Gyr --> 4000 = (SN * 10)/0.7 = 280
# Thus, we need snapshots numbers - 280, 420 and 600

def file_read(snapshot_number):
    # First we will extract the disk and the bulge particle of the corresponding snapshot number of the MW and M31
    # Add 000 to all snapshots
    ilbl = '000' + str(snapshot_number)
    # remove all but the last 3 digits
    ilbl = ilbl[-3:]
    # create filenames for both MW and M31
    MW_filename='%s_'%("MW") + ilbl + '.txt'
    M31_filename = '%s_'%("M31") + ilbl + '.txt'

    # Read data (assuming Read function is implemented elsewhere)
    MW_time, MW_total, MW_data = Read("/suhanisurana/"+MW_filename)
    print(MW_time.value)
    M31_time, M31_total, M31_data = Read("./"+M31_filename)

    # Extract disk (ptype = 2) and bulge (ptype = 3) particles
    MW_index = np.where((MW_data['type'] == 2) | (MW_data['type'] == 3))
    M31_index = np.where((M31_data['type'] == 2) | (M31_data['type'] == 3))


    # Store mass, positions, velocities, and type
    MW_particles = {
        'type': MW_data['type'][MW_index],  # Keep the actual type (2 or 3)
        'm': MW_data['m'][MW_index],
        'x': MW_data['x'][MW_index],
        'y': MW_data['y'][MW_index],
        'z': MW_data['z'][MW_index],
        'vx': MW_data['vx'][MW_index],
        'vy': MW_data['vy'][MW_index],
        'vz': MW_data['vz'][MW_index]
    }

    M31_particles = {
        'type': M31_data['type'][M31_index],  # Keep the actual type (2 or 3)
        'm': M31_data['m'][M31_index],
        'x': M31_data['x'][M31_index],
        'y': M31_data['y'][M31_index],
        'z': M31_data['z'][M31_index],
        'vx': M31_data['vx'][M31_index],
        'vy': M31_data['vy'][M31_index],
        'vz': M31_data['vz'][M31_index]
    }

    # Concatenate MW and M31 data
    # This will be later on used to access the mass and velocities directly
    combined_particles = {key: np.concatenate((MW_particles[key], M31_particles[key])) for key in MW_particles}

    output_filename = f"MWM31_{ilbl}.txt"
    # Write data to file in the same format
    with open(output_filename, 'w') as f:
        # This is the header
        f.write(f"Time   {MW_time.value}\n")
        f.write(f"Total      {len(combined_particles['m'])}\n")
        f.write("mass in 1e10,  x, y, z, in kpc and vx, vy, vz in km/s\n")
        f.write("#type, m, x, y, z, vx, vy, vz\n")

        # Write particle data
        for i in range(len(combined_particles['m'])):
            f.write(f"{combined_particles['type'][i]:f}  {combined_particles['m'][i]:f}  {combined_particles['x'][i]:f}  "
                    f"{combined_particles['y'][i]:f}  {combined_particles['z'][i]:f}  "
                    f"{combined_particles['vx'][i]:f}  {combined_particles['vy'][i]:f}  "
                    f"{combined_particles['vz'][i]:f}\n")

    print(f"Saved data to {output_filename}")
    return output_filename, combined_particles

In [6]:
def normalized_vectors(output_filename):
    """
    Compute the center of mass (RCOM) and velocity (VCOM) for the given snapshot file.
    Transform particle positions and velocities into the COM reference frame.

    Parameters:
    output_filename (str): Name of the file containing MW+M31 particle data.

    Returns:
    r (ndarray): Array of transformed positions relative to COM.
    v (ndarray): Array of transformed velocities relative to COM.
    """

    # Initialize CenterOfMass for disk particles (ptype = 2)
    COMD = CenterOfMass(output_filename, 2)

    # Compute COM of M31 using disk particles
    COMP = COMD.COM_P(0.1)
    COMV = COMD.COM_V(COMP[0],COMP[1],COMP[2])
 
    # Determine positions of disk particles relative to COM 
    xD = COMD.x - COMP[0].value 
    yD = COMD.y - COMP[1].value 
    zD = COMD.z - COMP[2].value 
    
    # total magnitude
    rtot = np.sqrt(xD**2 + yD**2 + zD**2)
    
    # Determine velocities of disk particles relatiev to COM motion
    vxD = COMD.vx - COMV[0].value 
    vyD = COMD.vy - COMV[1].value 
    vzD = COMD.vz - COMV[2].value 
    
    # total velocity 
    vtot = np.sqrt(vxD**2 + vyD**2 + vzD**2)
    
    # Arrays for r and v 
    r = np.array([xD,yD,zD]).T # transposed 
    v = np.array([vxD,vyD,vzD]).T
    return r, v

In [8]:
def RotateFrame(posI, velI):
    """
    Rotates the reference frame so that the angular momentum vector is aligned with the z-axis.
    
    Parameters:
    posI (ndarray): Nx3 array of particle positions.
    velI (ndarray): Nx3 array of particle velocities.
    
    Returns:
    pos (ndarray): Rotated positions.
    vel (ndarray): Rotated velocities.
    """

    # Compute total angular momentum vector
    L = np.sum(np.cross(posI, velI), axis=0)

    # Normalize angular momentum vector
    L_norm = L / np.linalg.norm(L)

    # Define z-axis unit vector
    z_norm = np.array([0, 0, 1])

    # Compute cross and dot products
    vv = np.cross(L_norm, z_norm)
    s = np.linalg.norm(vv)  # Magnitude of cross product
    c = np.dot(L_norm, z_norm)

    # Handle edge case: If already aligned with z-axis, no rotation needed
    if np.isclose(s, 0):  
        return posI, velI  # No rotation required

    # Rotation matrix (Rodrigues' rotation formula)
    I = np.eye(3)
    v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
    R = I + v_x + np.dot(v_x, v_x) * ((1 - c) / s**2)

    # Apply rotation
    pos = np.dot(R, posI.T).T
    vel = np.dot(R, velI.T).T

    return pos, vel

In [10]:
def compute_half_mass_radius(masses, radii):
    """
    Compute the half-mass radius given particle masses and radii.

    Parameters
    ----------
    masses : numpy array
        Array of particle masses.
    radii : numpy array
        Array of corresponding particle radii.

    Output
    ------
    half_mass_radius : float
        The radius within which half of the total mass is enclosed.
    """
    # Sort indices based on radii
    sorted_indices = np.argsort(radii)
    sorted_masses = masses[sorted_indices]
    sorted_radii = radii[sorted_indices]
    
    # Compute cumulative mass
    cumulative_mass = np.cumsum(sorted_masses)
    half_mass = np.sum(masses) / 2
    
    # Find the radius where half the mass is enclosed
    half_mass_radius = sorted_radii[np.searchsorted(cumulative_mass, half_mass)]
    return half_mass_radius

In [ ]:
output_filename, combined_particles = file_read(280)